# GOES Package
 - [Read NetCDF Data with Python]("https://towardsdatascience.com/read-netcdf-data-with-python-901f7ff61648#:~:text=To%20install%20with%20anaconda%20(conda,Then%20import%20netCDF4%20as%20nc%20.")
 - [using python to explore goes 16]("http://edc.occ-data.org/goes16/python/")
 

goes_tut data set, emailed from laura. Level 2 Multichannel formated data (ABI-L2-MCMIP) for the CONUS domain. 

In [1]:
import os 
import numpy as np
from datetime import datetime
import math
import sys 

import metpy 
import matplotlib.pyplot as plt
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray
from pyproj import Proj


from pymatreader import read_mat
import pandas as pd

In [2]:
# this is true so i am not in virtual environment
sys.prefix == sys.base_prefix


True

In [3]:
# get version of python
print(sys.version)

# this will list the system path ? 
#print(sys.path)

# this will list the path that jupiter is working in 
os.system("which python")
#/home/elenadworak/anaconda3/envs/mynewenv/bin/python

# this will list out all packages and their versions in the terminal 

#os.system("conda list")
# python 3.8.13 
# metpy 1.2.0

3.8.13 (default, Mar 28 2022, 11:38:47) 
[GCC 7.5.0]
/home/elenadworak/anaconda3/envs/mynewenv/bin/python


0

### Download netcdf data

In [4]:
ds = 'goes_tutorial.nc'
goes_tut_nc = nc.Dataset(ds)

goes_tut_nc
goes_tut_nc.variables
goes_tut_nc["CMI_C01"][:]
# method uses tostring() method 

cmi = goes_tut_nc["CMI_C01"]
cmi_values = cmi[:]
cmi_values

#print(goes_tut)

cmi = goes_tut_nc['CMI_C01'][:]

#int 1 --> int16 CMI_C16(y,x) the 16 channels
# goal to create real color image

FileNotFoundError: [Errno 2] No such file or directory: b'goes_tutorial.nc'

### Download lightning data 

In [ ]:
# A["Events"] is for the structure in matlab file

A = read_mat('data/2021-06-28-Bortnik.mat')
df_a = pd.DataFrame(A['Events'], columns = ["year", "month", "day", "hour", "minute", "second", "lat", "lon", "peak-current", "j",  "k", "l", "m"])

#select all columns except last four 
june28 = df_a.loc[:, ~df_a.columns.isin(["j",  "k", "l", "m"])]

B = read_mat('data/2021-06-29-Bortnik.mat')
df_b = pd.DataFrame(B['Events'], columns = ["year", "month", "day", "hour", "minute", "second", "lat", "lon", "peak-current", "j",  "k", "l", "m"])
june29 = df_b.loc[:, ~df_b.columns.isin(["j",  "k", "l", "m"])]

C = read_mat('data/2021-06-30-Bortnik.mat')
df_c = pd.DataFrame(C['Events'], columns = ["year", "month", "day", "hour", "minute", "second", "lat", "lon", "peak-current", "j",  "k", "l", "m"])
june30 = df_c.loc[:, ~df_c.columns.isin(["j",  "k", "l", "m"])]

D = read_mat('data/2021-07-01-Bortnik.mat')
df_d = pd.DataFrame(D['Events'], columns = ["year", "month", "day", "hour", "minute", "second", "lat", "lon", "peak-current", "j",  "k", "l", "m"])
july1 = df_d.loc[:, ~df_d.columns.isin(["j",  "k", "l", "m"])]


lightning = (june28, june29, june30, july1)

### Use X-array, much prettier

In [ ]:
# try using x_array 
# The opened file is assigned to goes_tut

FILE = ('goes_tutorial.nc')
goes_tut = xarray.open_dataset(FILE)
goes_tut

## NOAA Geostationary Operational Environmental Satellites (GOES) 16 & 17

GOES satellites (GOES-16 & GOES-17) provide continuous weather imagery and 
monitoring of meteorological and space environment data across North America.

In [ ]:
FILE = ('data/202118200/OR_ABI-L2-MCMIPC-M6_G17_s20211820016177_e20211820018561_c20211820019039.nc')
goes_tut = xarray.open_dataset(FILE)
goes_tut

path1 = 'data/202118200/'
file1 = os.listdir(path1)

for f in file1: 
    print(f)
    
# use a load command 

### Trying date time

time_coverage_start: 2021-06-28T00:01:17.1Z <br>
time_coverage_end: 2021-06-28T00:03:54.4Z <br>
timeline_id: ABI Mode 6 <br>
production_data_source: Realtime


In [ ]:
# Scan's start time, converted to datetime object
scan_start = datetime.strptime(goes_tut.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')

# Scan's end time, converted to datetime object
scan_end = datetime.strptime(goes_tut.time_coverage_end, '%Y-%m-%dT%H:%M:%S.%fZ')

# File creation time, convert to datetime object
file_created = datetime.strptime(goes_tut.date_created, '%Y-%m-%dT%H:%M:%S.%fZ')

print('Scan Start    : {}'.format(scan_start))
print('Scan End      : {}'.format(scan_end))
print('File Created  : {}'.format(file_created))
print('Scan Duration : {:.2f} minutes'.format((scan_end-scan_start).seconds/60))

## True Color RGB Recipe
Color images are a Red-Green-Blue (RGB) composite of three different channels. To make a “Natural True Color” image we assign the following channels as our R, G, and B values:

In [ ]:
# Load the three channels into appropriate R, G, and B variables

R = goes_tut['CMI_C02'][:]
G = goes_tut['CMI_C03'][:]
B = goes_tut['CMI_C01'][:]
print(R)

In [ ]:
# Apply range limits for each channel. RGB values must be between 0 and 1

R = np.clip(R, 0, 1)
G = np.clip(G, 0, 1)
B = np.clip(B, 0, 1)
print(R)

In [ ]:
# Apply a gamma correction to the image to correct ABI detector brightness
# takes everying in each array and raises it to 1/gamma 
# control image so it isn't too dark 

gamma = 2.2
R = np.power(R, 1/gamma)
G = np.power(G, 1/gamma)
B = np.power(B, 1/gamma)
print(R)

In [ ]:
# Calculate the "True" Green
# green "veggie" measures near-infared band sensitive to chlorophyll 
# will tone down green to get a true green 

G_true = 0.45 * R + 0.1 * G + 0.45 * B
G_true = np.clip(G_true, 0, 1)  # apply limits again, just in case.

In [ ]:
# First, plot each channel individually. The deeper the color means the satellite is observing more light in that channel. 
# Clouds appear white because they reflect lots of red, green, and blue light. 
# Notice that the land reflects a lot of “green” in the veggie channel because this channel is sensitive to the chlorophyll.

fig, ([ax1, ax2, ax3, ax4]) = plt.subplots(1, 4, figsize=(16, 3))

ax1.imshow(R, cmap='Reds', vmax=1, vmin=0)
ax1.set_title('Red', fontweight='bold')
ax1.axis('off')

ax2.imshow(G, cmap='Greens', vmax=1, vmin=0)
ax2.set_title('Veggie', fontweight='bold')
ax2.axis('off')

ax3.imshow(G_true, cmap='Greens', vmax=1, vmin=0)
ax3.set_title('"True" Green', fontweight='bold')
ax3.axis('off')

ax4.imshow(B, cmap='Blues', vmax=1, vmin=0)
ax4.set_title('Blue', fontweight='bold')
ax4.axis('off')

plt.subplots_adjust(wspace=.02)
plt.savefig("goes_color_subplots", dpi = 200)

The addition of the three channels results in a color image. Combine the three channels with a stacked array and display the image with imshow.

In [ ]:

# The RGB array with the raw veggie band
RGB_veggie = np.dstack([R, G, B])

# The RGB array for the true color image
RGB = np.dstack([R, G_true, B])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# The RGB using the raw veggie band
ax1.imshow(RGB_veggie)
ax1.set_title('GOES-16 RGB Raw Veggie', fontweight='bold', loc='left',
              fontsize=12)
ax1.set_title('{}'.format(scan_start.strftime('%d %B %Y %H:%M UTC ')),
              loc='right')
ax1.axis('off')

# The RGB for the true color image
ax2.imshow(RGB)
ax2.set_title('GOES-16 RGB True Color', fontweight='bold', loc='left',
              fontsize=12)
ax2.set_title('{}'.format(scan_start.strftime('%d %B %Y %H:%M UTC ')),
              loc='right')
ax2.axis('off')
plt.savefig("Rawveggie_truecolor", dpi = 200)

### Plot with Cartopy Geostationary Projection
The image above is not georeferenced. You can see the land and oceans, but we do have enough information to draw state and country boundaries. Use the metpy.io package to obtain the projection information from the file. Then use Cartopy to plot the image on a map. The GOES data and image is on a [geostationary projection ]

In [ ]:
# We'll use the `CMI_C02` variable as a 'hook' to get the CF metadata.
dat = goes_tut.metpy.parse_cf('CMI_C01')

geos = dat.metpy.cartopy_crs

# We also need the x (north/south) and y (east/west) axis sweep of the ABI data
x = dat.x
y = dat.y

In [ ]:
fig = plt.figure(figsize=(15, 12))

# Create axis with Geostationary projection
ax = fig.add_subplot(1, 1, 1, projection=geos)

# Add the RGB image to the figure. The data is in the same projection as the
# axis we just created.
ax.imshow(RGB, origin='upper',
          extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)

# Add Coastlines and States
ax.coastlines(resolution='50m', color='black', linewidth=0.25)
ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)

plt.title('GOES-16 True Color', loc='left', fontweight='bold', fontsize=15)
plt.title('{}'.format(scan_start.strftime('%d %B %Y %H:%M UTC ')), loc='right')

plt.show()

### Plot with Cartopy: Plate Carrée Cylindrical Projection

In [ ]:
fig = plt.figure(figsize=(8, 8))

pc = ccrs.PlateCarree()

ax = fig.add_subplot(1, 1, 1, projection=pc)
ax.set_extent([-114.75, -108.25, 36, 43], crs=pc)

ax.imshow(RGB, origin='upper',
          extent=(x.min(), x.max(), y.min(), y.max()),
          transform=geos,
          interpolation='none')

ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.add_feature(ccrs.cartopy.feature.STATES)

plt.title('GOES-16 True Color', loc='left', fontweight='bold', fontsize=15)
plt.title('{}'.format(scan_start.strftime('%d %B %Y %H:%M UTC ')), loc='right')

plt.show()

### Overlay Nighttime IR when dark
To view clouds in portions of the domain experiencing nighttime, we will overlay the clean infrared (IR) channel over the true color image.

First, open a file where the scan shows partial night area and create the true color RGB as before.

### Lightning overlay with goes 

In [ ]:

fig = plt.figure(figsize=(8, 8))

pc = ccrs.PlateCarree()

ax = fig.add_subplot(1, 1, 1, projection=pc)
ax.set_extent([-130, -100, 45, 65], crs=pc)


ax.imshow(RGB, origin='upper',
          extent=(x.min(), x.max(), y.min(), y.max()),
          transform=geos,
          interpolation='none')


ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.add_feature(ccrs.cartopy.feature.STATES)


points = (july1.loc[(july1.lon > -130) & (july1.lon < -100) & (july1.lat > 45) & (july1.lat < 65) & (july1["peak-current"] != 0) & (july1.hour == 0)])

plt.scatter(x = points["lon"], y = points["lat"], c = points["peak-current"] ** 0.001, cmap="OrRd", s = points["peak-current"] * 0.5, edgecolors="none", marker = 'o', alpha=0.5)


plt.title('GOES-17 True Color, lightning 0 UTC July 1', loc='left', fontweight='bold', fontsize=15)
plt.title('{}'.format(scan_start.strftime('%d %B %Y %H:%M UTC ')), loc='right')
plt.savefig("goes17_lightning", dpi = 200)
plt.show()


